In [1]:
#from transformer import Transformer 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import os
import gc
import random
import numpy as np
#import polars as pl
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
import fastai
from fastai.vision.all import *

from fastai.data.load import DataLoader
from fastai.learner import Learner
from torch.utils.data import Dataset
from fastai.metrics import rmse
from fastai.losses import *

import math
import copy

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#device = "cpu"


max_seq_len = 179 #103 #510 + 2  # +2 is added to cover start and end token of sequence This will change to 457 in private set  # Set the maximum sequence length to 100

data_preprocessing_done = 0
save_pkl = 0
load_pkl = 0
small_dataset = 0

#Training Data
if small_dataset == 1:
    
    #Uncomment this section when uploading the jupyter notebook
    '''
    df_read = pd.read_csv('/media/spartans/COMMON:/RNA_Project/train_data.csv')
    
    criteria_2A3 = (df_read['experiment_type'] == '2A3_MaP') & (df_read['SN_filter'] == 1)
    criteria_DMS = (df_read['experiment_type'] == 'DMS_MaP') & (df_read['SN_filter'] == 1)

    df_2A3_Filter1 = df_read[criteria_2A3]
    df_2A3_Filter1 = df_2A3_Filter1.head(4)

    df_DMS_Filter1 = df_read[criteria_DMS]
    df_DMS_Filter1 = df_DMS_Filter1.head(4)

   
    df = pd.concat([df_2A3_Filter1, df_DMS_Filter1], ignore_index=True)
    
    df.to_csv('2A3_DMS_With_Filter1_Train_Data.csv', index=False)
    #df_2A3_Filter1.to_csv('2A3_DMS_With_Filter1_Train_Data.csv', index=False)
    '''
    df = pd.read_csv('2A3_DMS_With_Filter1_Train_Data.csv')
    #df = pd.read_csv('DMS_Data.csv')
    #df = pd.read_csv('2A3_Data.csv')
else:
    df = pd.read_csv('/media/spartans/COMMON:/RNA_Project/train_data.csv')
    

#test_df = pd.read_csv('/media/spartans/COMMON:/RNA_Project/test_sequences.csv')
TEST_PATH = "/media/spartans/COMMON:/RNA_Project/test_sequences.csv"
SUBMISSION_PATH = "/media/spartans/COMMON:/RNA_Project/submission_1.csv"
SAMPLE_SUBMISSION_PATH = "/media/spartans/COMMON:/RNA_Project/sample_submission.csv"
MODEL_NAME = "Trained_Model.model_512_12_10_2023"

seed=69
os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)

os.environ['TORCH_USE_CUDA_DSA'] = str(1)


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(seed)

In [2]:



#criteria_2A3 = (df_read['experiment_type'] == '2A3_MaP') & (df_read['SN_filter'] == '1')
#criteria_DMS = (df_read['experiment_type'] == 'DMS_MaP') & (df_read['SN_filter'] == '1')

#criteria_2A3 = ((df_read['experiment_type'] == 'DMS_MaP')  & df_read['SN_filter'] == 1 )
#df_2A3_Filter1 = df_read[criteria_2A3]
#df_2A3_Filter1
#df
#df.iloc[2,0:300].values
# Set display options to show all columns and rows
#pd.set_option('display.max_columns', None)
#df

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-1, -2)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        #print(output)
        return output

In [4]:
class PositionWiseFeedForward1(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))
    
class PositionWiseFeedForward(nn.Module):

    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionWiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        #print(f"x after first linear layer: {x.size()}")
        x = self.relu(x)
        #print(f"x after activation: {x.size()}")
        x = self.dropout(x)
        #print(f"x after dropout: {x.size()}")
        x = self.linear2(x)
        #print(f"x after 2nd linear layer: {x.size()}")
        return x


In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        #print(x.size())
        #print(self.pe[:, :x.size(1)].size())
        #print((x + self.pe[:, :x.size(1)]).size())
        return x + self.pe[:, :x.size(1)]
    

class PositionalEncoding1(nn.Module):

    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self, x):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = torch.arange(self.max_sequence_length).reshape(self.max_sequence_length, 1)
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        print("PE size", PE.size())
        print("x size", x.size())
        print("PE x size", PE[:, :x.size(1)].size())
        print("stacked dim", stacked.size())
        return x + PE[:, :x.size(1)]
    

In [6]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [7]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x
    
    


In [8]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        
        
        #self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        
        self.positional_encoding = PositionalEncoding(d_model, 179)


        #self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        #self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        
        
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=d_ff,
                dropout=0.1, activation=nn.GELU(), batch_first=True, norm_first=True), num_layers)
        
        
        
        #self.transformer_decoder = nn.TransformerDecoder(
        #    nn.TransformerDecoderLayer(d_model=d_model, nhead=num_heads),
        #    num_layers=num_layers
        #)
        
        
        
        #self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.gelu = nn.GELU()
        self.fc1 = nn.Linear(max_seq_length,max_seq_length)
        
        # Linear layer for regression output
        #self.regressor = nn.Linear(d_model, max_seq_length)
        self.regressor = nn.Linear(d_model, max_seq_length, dtype=torch.float32)  # Adjusted for float32


    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        #print("forward ",src, src.size())
        #print("forward --", tgt, tgt.size())
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        #print("src mask --", src_mask, src_mask.size(), tgt_mask, tgt_mask.size())
        
        
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        #print("Src_embedded size", src_embedded, src_embedded.size())
        #print("tgt size ", tgt.size())
        #print("TGT  = ", tgt)
        #tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))
        #print("tgt_embedded size", tgt_embedded.size())

        #enc_output = src_embedded
        #for enc_layer in self.encoder_layers:
        #    enc_output = enc_layer(enc_output, src_mask)
            
        enc_output = self.transformer(src_embedded)

        #dec_output = tgt_embedded
        #for dec_layer in self.decoder_layers:
        #    dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)
        
        
        ################
        # Global average pooling along the sequence dimension
        tgt = enc_output.mean(dim=1)

        # Apply GELU activation to the output
        tgt = nn.functional.gelu(tgt)

        # Regression output with sigmoid activation to clip between 0 and 1
        final_out = torch.sigmoid(self.regressor(tgt))
        ################
        
        
        # Expand the dimensions to create an initial input for the decoder
        #x = enc_output.unsqueeze(1).expand(-1, self.output_sequence_length, -1)


        #output = self.fc(dec_output)
        #print("Size OUTPUT",output.size())
        
        #xx = self.gelu(self.fc(dec_output))
        #print("Size xx", xx.size())
        #final_out = self.fc1(xx)
        #print("Size final_out", final_out, final_out.size())
        return final_out
        #include here FC layers with output dimension = input dimension of the sequence (considering padding this will be const number)
        #return output

In [9]:
#transformer




In [10]:
#src_data
#src_data.size()

In [11]:
#src_data.size()
#src_data = torch.randint(1, src_vocab_size, (64, max_seq_length)).to(device)  # (batch_size, seq_length)

In [12]:
#df


In [13]:
#where reactivity is NaN,
# have Nan replaced with average of all numbers in the sequence.

#Map sequence letters to numbers.

#Looking at Reads and Signal To Noise ..they appear to be somewhat coorelated.

#Create Dataset function


#In contrastive loss model... you can train based on#
#in a sequence which part is 2D fold and which part is 3D fold
#See notebook https://www.kaggle.com/code/something4kag/ribonanza-3d-coords-prep
#if it can be helpful
# Also analyze and which position in the sequence the fold can occur and which fold ended up happening based on the data

#another method to formulate contrastive learning model is by deferentiating sequences with low SNR and hence reactivity value will not be with high confidence.



In [14]:

#In RNA, the most common base pairings you'll find are between the following nucleotide bases:

#Adenine (A) and Uracil (U): A forms base pairs with U. This is a fundamental pairing and is commonly seen in RNA molecules, particularly in single-stranded regions. It is important for the stability of stem-loop structures and is a key component of RNA secondary structure.

#Guanine (G) and Cytosine (C): G forms base pairs with C, just as it does in DNA. This pairing is less common in RNA secondary structure but is still important for certain RNA molecules. It may be more prevalent in the context of ribozymes and catalytic RNA.

#While A-U and G-C are the primary base pairings in RNA, it's essential to understand that RNA can also exhibit non-canonical or non-standard base pairings, especially in more complex RNA structures. These non-canonical pairings can involve different combinations of A, U, G, and C and are often seen in tertiary structures or specialized RNA molecules with specific functions.

#The prevalence of A-U and G-C base pairings in an RNA molecule can vary depending on its sequence and function. For instance, regions that need to form stable secondary structures often rely on A-U pairings, while regions involved in catalytic activities may include G-C pairs. RNA structures are diverse and can exhibit a wide range of base pairing interactions to achieve their biological functions.



#So a contrastive loss can be formed here where
#AU pair GC pair are strong positive - stable
#AG, UC are less stable
#UG, UA, CG pair are unstable so negative .
#
#In RNA, the stable base pairs among the four nucleotide bases (A, G, U, and C) follow the standard Watson-Crick base-pairing rules. Here are the stable base pairs among these bases:

#GC pairs are more stable than AU pair


#So you can tokenzie these pairs in the transformer model.

In [15]:
#Formulate Convolution NN or Dense Net which takes SNR, Reactivity error and feeds into the final layer of Transformer network
# As an example
#Lowest Reactivity Error and High SNR are  strong positives
# Lowest SNR are strong negative.





#sequence length range from 115 to 206 in train dataset.
#in final it will range from 207 to 457

In [16]:
#In positional encoding
#pos means the position of the word in the sequence.
#i means the index for that particular word vector (index of the dimension)
#dmodel is the dimension for e.g 512 from the example


#For 2 experiment types i.e. DMS and 2A3 you can have start and end  token in a sequence for which data is available.
#.e. start_DMS, end_DMS, start_2A3, end_2A3.




In [17]:
#df

In [18]:
#df.iloc[:,150:213]


In [ ]:
if data_preprocessing_done == 0:
    #Assigning reactivity Columns to react_columns dataframe and replacing NaN with 0.000 

    #del(mean_reactivity)
    #del(react_columns_New)
    #del(react_columns)
    
    #react_columns = df.iloc[:,7:213].fillna(0.000000)
    
    react_columns = df.iloc[:,33:136].fillna(0.000000)


    #Find mean of reactivity per row
    #df['mean'] = df.mean(axis=1)
    #react_columns = react_columns.applymap(lambda x: 0.0 if x < 0.0 else x)
    #react_columns = react_columns.applymap(lambda x: 0 if isinstance(x, (float, int)) and x < 0 else x)
    
    react_columns = react_columns.applymap(lambda x: -1*x if isinstance(x, (float, int)) and x < 0 else x)
    react_columns = react_columns.applymap(lambda x: 1.0 if isinstance(x, (float, int)) and x > 1.0 else x)

    #react_columns_mean = react_columns.mean(axis=1)
    #Free up memory
    #del(react_columns)

    #Now replace NaN in react_columns with mean_reactivity
    #react_columns_New = df.iloc[:,7:213].fillna(mean_reactivity)

/tmp/ipykernel_154709/1617856248.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  react_columns = react_columns.applymap(lambda x: -1*x if isinstance(x, (float, int)) and x < 0 else x)


In [ ]:

#if data_preprocessing_done == 0:
#    react_columns = df.iloc[:,7:213] #You may have to capture those indices as per sequence length i.e. when it extends to 457

    # Replace NaN values in each row with values from replace_values array
#    for index, row in tqdm(react_columns.iterrows(), total=len(react_columns), desc="Processing rows"):
#        react_columns.loc[index] = row.fillna(react_columns_mean[index])
    
#    react_columns = react_columns.applymap(lambda x: 0 if isinstance(x, (float, int)) and x < 0 else x)


        
        
        

In [ ]:
#react_columns_New = df.iloc[:,7:213].fillna(react_columns['mean_reactivity'])
#react_columns_New
#react_columns.columns


#for column_name in tqdm(react_columns.columns, total=len(react_columns.columns), desc="Processing"):
#    df[column_name] = react_columns[column_name]
    
#react_columns['sequence'] = df['experiment_type'] + 'start' + df['sequence']
#react_columns['sequence'] = react_columns['sequence'] + 'end' + df['experiment_type']






In [ ]:
seq_map1 = {'A':1.0,
           'C':2.0,
           'G':3.0,
           'U':4.0, 
           'M':5.0,  #Start token of DMS exp seq
           'N':6.0,  #End token of DMS exp seq
           'T':7.0,  #Start token of 2A3 exp seq
           'X':8.0,  #End token of 2A3 exp seq
           'Z':0.0   #Padding token
          }


seq_map = {'A':1,
           'C':2,
           'G':3,
           'U':4, 
           'M':5,  #Start token of DMS exp seq
           'N':6,  #End token of DMS exp seq
           'T':7,  #Start token of 2A3 exp seq
           'X':8,  #End token of 2A3 exp seq
           'Z':0   #Padding token
          }
#for s in df.at[0,'sequence']:
#    print(seq_map[s])
#df

#test_cell = [seq_map[s] for s in df.at[0, 'sequence']]
#print(test_cell)

#len(df.at[0, 'sequence'])
# Create a list of random data samples (for demonstration)
#max_seq_len = 206 #This will change to 457 in private set  # Set the maximum sequence length to 100


In [ ]:
# Iterate over rows and replace a specific value in all columns

#seq = [self.seq_map[s] for s in seq]
#for index, row in df.iterrows():
# Create a list of random data samples (for demonstration)
#max_seq_len = 206 #This will change to 457 in private set  # Set the maximum sequence length to 100
if data_preprocessing_done == 0:
    df_sq = copy.deepcopy(df['sequence']) #saving a backup of sequences.
    #react_columns['maped_sequence'] = [' ']*len(react_columns)
    #for index, row in tqdm(df.iterrows(), total=len(df.rows), desc="Processing rows"):
    for index, row in tqdm(react_columns.iterrows(), total=len(react_columns), desc="Processing rows"):
        if df.at[index, 'experiment_type'] == "2A3_MaP":
            concat_seq = 'T' + df.at[index, 'sequence'] + 'X'
        if df.at[index, 'experiment_type'] == "DMS_MaP":
            concat_seq = 'M' + df.at[index, 'sequence'] + 'N'
        padding = "Z"*(max_seq_len - 2 - len(df.at[index, 'sequence']))
        concatenated_seq = concat_seq + padding
        df.at[index, 'sequence'] = [seq_map[s] for s in concatenated_seq]
        #react_columns.at[index, 'maped_sequence'] = [seq_map[s] for s in concatenated_seq]


In [ ]:

    
if data_preprocessing_done == 0:
    #react_columns['reactivity_0207'] = copy.deepcopy(react_columns['reactivity_0206'])
    #react_columns['reactivity_0208'] = copy.deepcopy(react_columns['reactivity_0206'])
    
    for iii in range(130,206):
        react_col_name = "reactivity_0" + str(iii)
        react_columns[react_col_name] = copy.deepcopy(react_columns['reactivity_0129'])
        
    #replicated_columns = [react_columns['reactivity_0206'].repeat(2)]
    #react_columns = pd.concat([react_columns, replicated_columns] , axis=1, ignore_index=True)
    #print(react_columns)
    react_columns['mapped_sequence'] = copy.deepcopy(df['sequence'])
    #del react_columns['reactivity_0207']
    #del react_columns['reactivity_0208']
if save_pkl == 1:
    react_columns.to_pickle("react_columns.pkl")

if load_pkl == 1:
    react_columns = pd.read_pickle("react_columns.pkl")

In [ ]:
#react_columns.select_dtypes(exclude="NaN")
#df[~df['COLUMN1'].str.contains('TOTAL')]
#react_columns.iloc[~react_columns[:,1].str.contains('NaN')]

#react_columns.iloc[:,1]

#data1 = [[1, 1, 2], [6, 4, 2], [4, 2, 1], [4, 2, 3]]

#daf = pd.DataFrame(data1)
#print(data1)
#print(daf.mean())
#react_columns['sequence'][150:170][160]
react_columns

In [ ]:
# Define a custom dataset class for your data
class CustomDataset(Dataset):
    def __init__(self, data, max_seq_len):
        self.data = data
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #src_sequence, trg_sequence = [0,1] #self.data[idx]
        #src_sequence = self.data['reactivity_0001']
        #trg_sequence = self.data['mapped_sequence']
        #list_src_trg_seq = self.data.loc[idx].apply(lambda row: row.tolist(), axis=1)
        #merged_values = ','.join(map(str, df.loc[row_index_to_merge][:-1]))
        #list_src_trg_seq = ','.join(map(str, self.data.loc[row_index_to_merge][:-1]))
        #src_sequence = list_src_trg_seq[0:-1]
        #srg_sequence = list_src_trg_seq[-1]
        
        #trg_sequence = ','.join(map(str, self.data.loc[idx][:-1]))
        #src_sequence = [','.join(map(str, self.data.loc[idx][-1]))]
        
        trg_sequence = torch.tensor(self.data.iloc[idx, :].tolist()[:-1])
        src_sequence = torch.tensor(self.data.iloc[idx, :].tolist()[-1])
        
        #print("Size src seq ", src_sequence.size())
        #print("Size trg seq ", trg_sequence, trg_sequence.size())
        

        #trg_sequence = self.data.iloc[idx, :].totensor()[:-1]
        #src_sequence = self.data.iloc[idx, :].totensor()[-1]
        
        # Pad sequences with zeros to match the length of the longest sequence in each batch
        #max_len = max(len(src_sequence), len(trg_sequence), self.max_seq_len)
        #src_sequence += [0] * (max_len - len(src_sequence))
        #trg_sequence += [0] * (max_len - len(trg_sequence))
        #print("Index ", idx, " SRC--", src_sequence, " TRG--", trg_sequence)
        #print("SRC LEN ", len(src_sequence), " TRG LEN ", len(trg_sequence))
        return src_sequence, trg_sequence



#data = []
#for _ in range(5):
#    src_sequence = [random.randint(1, 100) for _ in range(random.randint(5, max_seq_len))]
#    trg_sequence = [random.randint(1, 100) for _ in range(random.randint(5, max_seq_len))]
#    data.append((src_sequence, trg_sequence))

#print("src", len(src_sequence))
#print("trg", len(trg_sequence))

#list_src_trg_seq = react_columns.apply(lambda row: row.tolist(), axis=1)

    
# Create a DataLoader for batching and shuffling
batch_size = 1
#batch_size = 3


# Define the size of the training set
dataset_size = len(react_columns)
train_size = int(0.9 * dataset_size)
test_size = dataset_size - train_size

# Create indices for training and testing sets
#indices = list(range(dataset_size))
#train_indices, test_indices = random_split(indices, [train_size, test_size])
#print(train_indices)
#print(test_indices)
custom_dataset = CustomDataset(react_columns, max_seq_len)
train_indices, test_indices = random_split(custom_dataset, [train_size, test_size])
train_dataloader = DataLoader(train_indices, batch_size=batch_size)
test_dataloader = DataLoader(test_indices, batch_size=batch_size)

#train_dataloader = DataLoader(custom_dataset, batch_size=batch_size)
#test_dataloader = DataLoader(custom_dataset, batch_size=batch_size)


#data = DataLoader(custom_dataset)
  
# Example usage in the training loop (as previously shown)
#for batch in dataloader:
#    print("Inside DataLoader Func")
#    src, trg = batch
#    print("SRC -", src, src.size())
#    print("TRG -", trg, trg.size())
    # ...

    

In [ ]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"
#print(device)
src_vocab_size = 512
#tgt_vocab_size = 5000
tgt_vocab_size = 512
d_model = 256
num_heads = 8
num_layers = 6
d_ff = 1024
max_seq_length = max_seq_len
dropout = 0.1



src_vocab_size = 5000
#tgt_vocab_size = 5000
tgt_vocab_size = 103
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = max_seq_len
dropout = 0.1

# Generate random sample data
#src_data = torch.randint(1, src_vocab_size, (64, max_seq_length)).to(device)  # (batch_size, seq_length)
#tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length)).to(device)  # (batch_size, seq_length)

transformer_model = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout).to(device)

#transformer = nn.Transformer()

In [ ]:
'''
def los_func(src_data, tgt_data, tgt_vocab_size):
    transformer_model.eval()
    output = transformer_model(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    transformer_model.train()
    return loss

for epoch in range(3):
    for src_data, tgt_data in train_dataloader:          
        src_data, tgt_data = src_data.to(device), tgt_data.to(device,  dtype=torch.long)
        learn = Learner(data, transformer_model, loss_func=los_func(src_data, tgt_data,tgt_vocab_size)).to_fp16() 
        learn.fit_one_cycle(32, lr_max=5e-4, wd=0.05, pct_start=0.02)
        torch.save(learn.model.state_dict(),"Model_Trained.model")
        gc.collect()
'''

In [ ]:
#criterion = nn.CrossEntropyLoss(ignore_index=0)
#criterion = nn.MSELoss()

#criterion = nn.L1Loss(size_average=None, reduce=None, reduction='mean')



def criterion(pred, target):
    loss = F.l1_loss(pred, target.clip(0,1), reduction='none')
    loss = loss[~torch.isnan(loss)].mean()
    return loss



optimizer = optim.Adam(transformer_model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)




training_losses = []  # to store training losses
validation_losses = []
prev_average_val_loss = 10000000000
list_of_avg_loss = []
epochss = []

num_epochs = 2

for epoch in range(num_epochs):
    epoch_loss = 0.0  # accumulate loss for the entire epoch
    val_loss = 0.0
    
    transformer_model.train()
    #for src_data, tgt_data in train_dataloader:
    print("IN TRAIN")
    for batch, dat in enumerate(tqdm(train_dataloader, desc="Processing batches", unit="batch"), 1):

        #print("TARGET DATA - TRAINING ", tgt_data, tgt_data.size())
        
        src_data, tgt_data = dat[0].to(device, dtype=torch.long), dat[1].to(device, dtype=torch.float32)
        
        #print("TARGET DATA - TRAINING ", tgt_data, tgt_data.size())
        #print("SRC DATA - TRAINING ",  src_data, src_data.size())

        optimizer.zero_grad()
        output = transformer_model(src_data, tgt_data)
        #print("OUPUT DATA - TRAINING ",  output, output.size())
        #loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
        loss = criterion(output, tgt_data)
        
        loss.backward()
        optimizer.step()
          
        epoch_loss += loss.item()
            
    
    transformer_model.eval()
    with torch.no_grad():
            #for src_data, tgt_data in test_dataloader:
            print("IN VAL")
            for batch, dat in enumerate(tqdm(test_dataloader, desc="Processing batches", unit="batch"), 1):
                #This should be validation set data.
                src_data, tgt_data = dat[0].to(device, dtype=torch.long), dat[1].to(device, dtype=torch.float32)
                #src_data, tgt_data = src_data.to(device, dtype=torch.long), tgt_data.to(device, dtype=torch.float32)
                #print("src ", src_data)
                #print("src size", src_data.size())
                #print("tgt ", tgt_data)
                #print("tgt size", tgt_data.size())
                #output = transformer_model(src_data, tgt_data[:, :-1])
                output = transformer_model(src_data, tgt_data)
                #print("Target Data - VALIDATION ", tgt_data, tgt_data.size())
                #print("Predicted Target Data - Validation " , output, output.size())
                
                
                #output[output < 0] = 0.0
                #output[output > 1] = 1.0
                loss = criterion(output, tgt_data)
                #print(f"Epoch: {epoch+1}, Validation Loss: {loss.item()}")
                val_loss += loss.item()
        

    #torch.save(transformer_model, "Trained_Model.model")

    average_epoch_loss = epoch_loss / len(train_dataloader)
    training_losses.append(average_epoch_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_epoch_loss:.4f}')
    
    average_val_loss = val_loss / len(test_dataloader)
    validation_losses.append(average_val_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {average_val_loss:.4f}')
    epochss.append(epoch)
    
    list_of_avg_loss.append(average_val_loss)
    #Save best model Every Epoch
    if average_val_loss < prev_average_val_loss:
        torch.save(transformer_model.state_dict(), MODEL_NAME)
        
        print("Model Saved because ")
        print(f'Current average validation loss {average_val_loss:.4f} is less than previous average validation loss {prev_average_val_loss:.4f}')
    prev_average_val_loss = min(list_of_avg_loss)

# Plotting the training loss
plt.plot(epochss, training_losses, label='Training Loss')
plt.plot(epochss, validation_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.show()

In [ ]:
transformer_model

In [ ]:
transformer_model.load_state_dict(torch.load(MODEL_NAME,map_location=torch.device(device)))
transformer_model.eval()

In [ ]:


#Read TEST Sequence
test_df = pd.read_csv(TEST_PATH)

In [ ]:

test_df['DMS_sequence'] = copy.deepcopy(test_df['sequence'])
test_df['2A3_sequence'] = copy.deepcopy(test_df['sequence'])


test_df_sq = copy.deepcopy(test_df['sequence']) #saving a backup of sequences.
#react_columns['maped_sequence'] = [' ']*len(react_columns)
#for index, row in tqdm(df.iterrows(), total=len(df.rows), desc="Processing rows"):
for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing rows"):
    
    padding = "Z"*(max_seq_len - 2 - len(test_df.at[index, 'sequence']))
    form_DMS_seq = 'M' + test_df.at[index, 'sequence'] + 'N' + padding
    form_2A3_seq = 'T' + test_df.at[index, 'sequence'] + 'X' + padding
    

    test_df.at[index, 'DMS_sequence'] = [seq_map[s] for s in form_DMS_seq]
    test_df.at[index, '2A3_sequence'] = [seq_map[s] for s in form_2A3_seq]
    
    


In [ ]:
sample_df = pd.read_csv(SAMPLE_SUBMISSION_PATH)


In [ ]:
#print(tgt_sequence.size())

#test_df['id_max'][0]
#print(test_df)
sample_df['reactivity_DMS_MaP'] = sample_df['reactivity_DMS_MaP'].astype(float)
sample_df['reactivity_2A3_MaP'] = sample_df['reactivity_DMS_MaP'].astype(float)
print("DONE")

In [ ]:
tgt_sequence = torch.rand((1, 179), dtype=torch.float32).to(device=device)


In [ ]:

#print("seq split", seq_split)
element_to_prepend_2a3 = torch.tensor([7]).to(device=device, dtype=torch.long)
element_to_postpend_2a3 = torch.tensor([8]).to(device=device, dtype=torch.long)

element_to_prepend_dms = torch.tensor([5]).to(device=device, dtype=torch.long)
element_to_postpend_dms = torch.tensor([6]).to(device=device, dtype=torch.long)


for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing rows"):
    input_sequence = torch.tensor([test_df.at[index, 'DMS_sequence']]).to(device=device, dtype=torch.long)
    #print(input_sequence, input_sequence.shape)
    seq_length = len(test_df.at[index, 'sequence'])
    #print(f'{seq_length}')
    #if seq_length != 177:
        #print(f' seq length {seq_len}')
        #id_minn = test_df.at[index, 'id_min']
        #id_maxx = test_df.at[index, 'id_max']
        #print(f'index {index}, seq len {seq_length}, id_min {id_minn},  id_max {id_maxx} ')
    #First check what is the sequence length. If it is less than or equal to 206 then process it straight.
    
    id_minn = test_df.at[index, 'id_min']
    id_maxx = test_df.at[index, 'id_max']
    #print(f'index {index}, seq len {seq_length}, id_min {id_minn},  id_max {id_maxx} , {input_sequence.size()} {tgt_sequence.size()}')
    
    
    
    
    seq_split = seq_length/179


    if seq_split > 1 and seq_split < 2:
        in_sq_0 = input_sequence[0][1:178] 
        in_sq_0_2a3 = torch.cat((element_to_prepend_2a3, in_sq_0, element_to_postpend_2a3))
        out_2a3_0 = transformer_model(in_sq_0_2a3,tgt_sequence).to(dtype=torch.float16)

        in_sq_0_dms = torch.cat((element_to_prepend_dms, in_sq_0, element_to_postpend_dms))
        out_dms_0 = transformer_model(in_sq_0_dms,tgt_sequence).to(dtype=torch.float16)



        in_sq_1 = input_sequence[0][179:-1]

        in_sq_1_2a3 = torch.cat((element_to_prepend_2a3, in_sq_1, element_to_postpend_2a3))



        padding = (179 - len(in_sq_1_2a3))
        zeros_to_append = torch.zeros(padding).to(device=device, dtype=torch.long)
        in_sq_1_2a3 = torch.cat((in_sq_1_2a3, zeros_to_append))

        out_2a3_1 = transformer_model(in_sq_1_2a3,tgt_sequence).to(dtype=torch.float16)

        in_sq_1_dms = torch.cat((element_to_prepend_dms, in_sq_1, element_to_postpend_dms))    
        in_sq_1_dms = torch.cat((in_sq_1_dms, zeros_to_append))
        out_dms_1 = transformer_model(in_sq_1_dms,tgt_sequence).to(dtype=torch.float16)

        iiii = 0
        for iii in range(id_minn, id_minn + 178):
            sample_df.at[iii, 'reactivity_2A3_MaP'] = out_2a3_0.tolist()[0][iiii]  
            sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms_0.tolist()[0][iiii]
            iiii = iiii + 1  

        iiii = 0
        for iii in range(id_minn + 179, id_maxx):
            sample_df.at[iii, 'reactivity_2A3_MaP'] = out_2a3_1.tolist()[0][iiii]  
            sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms_1.tolist()[0][iiii]
            iiii = iiii + 1 

        #print("1 here")


    if seq_split > 2:
        in_sq_0 = input_sequence[0][1:178] 
        in_sq_0 = torch.cat((element_to_prepend_2a3, in_sq_0, element_to_postpend_2a3))

        out_2a3_0 = transformer_model(in_sq_0,tgt_sequence).to(dtype=torch.float16)


        in_sq_1 = input_sequence[0][179:356]
        in_sq_1 = torch.cat((element_to_prepend_2a3, in_sq_1, element_to_postpend_2a3))

        out_2a3_1 = transformer_model(in_sq_1,tgt_sequence).to(dtype=torch.float16)


        in_sq_2 = input_sequence[0][357:-1]
        in_sq_2 = torch.cat((element_to_prepend_2a3, in_sq_2, element_to_postpend_2a3)) 


        padding = (179 - len(in_sq_2))
        zeros_to_append = torch.zeros(padding).to(device=device, dtype=torch.long)
        in_sq_2 = torch.cat((in_sq_2, zeros_to_append))

        out_2a3_2 = transformer_model(in_sq_2,tgt_sequence).to(dtype=torch.float16)

        ##############
        in_sq_0 = input_sequence[0][1:178] 
        in_sq_0 = torch.cat((element_to_prepend_dms, in_sq_0, element_to_postpend_dms))

        out_dms_0 = transformer_model(in_sq_0,tgt_sequence).to(dtype=torch.float16)


        in_sq_1 = input_sequence[0][179:356]
        in_sq_1 = torch.cat((element_to_prepend_dms, in_sq_1, element_to_postpend_dms))

        out_dms_1 = transformer_model(in_sq_1,tgt_sequence).to(dtype=torch.float16)


        in_sq_2 = input_sequence[0][357:-1]
        in_sq_2 = torch.cat((element_to_prepend_dms, in_sq_2, element_to_postpend_dms)) 


        padding = (179 - len(in_sq_2))
        zeros_to_append = torch.zeros(padding).to(device=device, dtype=torch.long)
        in_sq_2 = torch.cat((in_sq_2, zeros_to_append))

        out_dms_2 = transformer_model(in_sq_2,tgt_sequence).to(dtype=torch.float16)

        print(f'{out_dms_0.size()} {out_dms_1.size()} {out_dms_2.size()} {id_minn} {id_maxx} {id_maxx - id_minn}')

        iiii = 0
        for iii in range(id_minn, id_minn + 178):
            sample_df.at[iii, 'reactivity_2A3_MaP'] = out_2a3_0.tolist()[0][iiii]  
            sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms_0.tolist()[0][iiii]
            iiii = iiii + 1  

        iiii = 0
        for iii in range(id_minn + 179, id_minn + 179 + 178):
            sample_df.at[iii, 'reactivity_2A3_MaP'] = out_2a3_1.tolist()[0][iiii]  
            sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms_1.tolist()[0][iiii]
            iiii = iiii + 1  

        iiii = 0
        for iii in range(id_minn + 179 + 179, id_maxx):
            sample_df.at[iii, 'reactivity_2A3_MaP'] = out_2a3_2.tolist()[0][iiii]  
            sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms_2.tolist()[0][iiii]
            iiii = iiii + 1  
        #print("here")
        
        
    
    
    
    
    
    
    
    
    
    
    
    if seq_split <= 1:
        #out_dms = []
        out_dms = transformer_model(input_sequence,tgt_sequence).to(dtype=torch.float16)
        out_dms[out_dms < 0] = 0.0
        out_dms[out_dms > 1] = 1.0

        out_dms = out_dms.tolist()   
        #iiii = 0
        #for iii in range(id_minn, id_maxx): 
        #    sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms[0][iiii]
        #    sample_df.at[iii, 'id'] = iii
        #    iiii = iiii + 1


        #out_2a3 = []
        input_sequence = torch.tensor([test_df.at[index, '2A3_sequence']]).to(device=device, dtype=torch.long)
        out_2a3 = transformer_model(input_sequence,tgt_sequence).to(dtype=torch.float16)
        out_2a3[out_2a3 < 0] = 0.0
        out_2a3[out_2a3 > 1] = 1.0
        out_2a3 = out_2a3.tolist()
        #out_2a3[:seq_length]
        #print(out_2a3)
        #sample_df[id_minn:id_maxx]['reactivity_2A3_MaP'] = pd.DataFrame({'reactivity_2A3_MaP': out_2a3[0][:seq_length]}, index=range(id_minn, id_minn + seq_length))
        #print(out_dms[0][25:25+176])
        #print(out_2a3[0][25:25+176])

        iiii = 0
        for iii in range(id_minn, id_maxx):
            sample_df.at[iii, 'reactivity_2A3_MaP'] = out_2a3[0][iiii]  
            sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms[0][iiii]
            iiii = iiii + 1  
        
    
    
    #print(sample_df['reactivity_2A3_MaP'].iloc[0])
    #print(sample_df['reactivity_DMS_MaP'].iloc[0])
    #break

    #if index==2:
    #    break
    
    
    
#sample_df   


In [ ]:
'''
input_sequence = torch.tensor([test_df.at[335823, '2A3_sequence']]).to(device=device, dtype=torch.long)
print(input_sequence[0][:179], input_sequence[0][:179].size(), id_minn, id_maxx, id_maxx-id_minn)


seq_length = len(test_df.at[335823, 'sequence'])

id_minn = test_df.at[335823, 'id_min']
id_maxx = test_df.at[335823, 'id_max']

seq_split = seq_length/179
print("seq split", seq_split)
element_to_prepend_2a3 = torch.tensor([7]).to(device=device, dtype=torch.long)
element_to_postpend_2a3 = torch.tensor([8]).to(device=device, dtype=torch.long)

element_to_prepend_dms = torch.tensor([5]).to(device=device, dtype=torch.long)
element_to_postpend_dms = torch.tensor([6]).to(device=device, dtype=torch.long)

if seq_split > 1 and seq_split < 2:
    in_sq_0 = input_sequence[0][1:178] 
    in_sq_0_2a3 = torch.cat((element_to_prepend_2a3, in_sq_0, element_to_postpend_2a3))
    out_2a3_0 = transformer_model(in_sq_0_2a3,tgt_sequence).to(dtype=torch.float16)
    
    in_sq_0_dms = torch.cat((element_to_prepend_dms, in_sq_0, element_to_postpend_dms))
    out_dms_0 = transformer_model(in_sq_0_dms,tgt_sequence).to(dtype=torch.float16)
    
    
    
    in_sq_1 = input_sequence[0][179:-1]
    
    in_sq_1_2a3 = torch.cat((element_to_prepend_2a3, in_sq_1, element_to_postpend_2a3))

    
    
    padding = (179 - len(in_sq_1_2a3))
    zeros_to_append = torch.zeros(padding).to(device=device, dtype=torch.long)
    in_sq_1_2a3 = torch.cat((in_sq_1_2a3, zeros_to_append))
    
    out_2a3_1 = transformer_model(in_sq_1_2a3,tgt_sequence).to(dtype=torch.float16)
    
    in_sq_1_dms = torch.cat((element_to_prepend_dms, in_sq_1, element_to_postpend_dms))    
    in_sq_1_dms = torch.cat((in_sq_1_dms, zeros_to_append))
    out_dms_1 = transformer_model(in_sq_1_dms,tgt_sequence).to(dtype=torch.float16)
    
    iiii = 0
    for iii in range(id_minn, id_minn + 178):
        sample_df.at[iii, 'reactivity_2A3_MaP'] = out_2a3_0.tolist()[0][iiii]  
        sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms_0.tolist()[0][iiii]
        iiii = iiii + 1  
    
    iiii = 0
    for iii in range(id_minn + 179, id_maxx):
        sample_df.at[iii, 'reactivity_2A3_MaP'] = out_2a3_1.tolist()[0][iiii]  
        sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms_1.tolist()[0][iiii]
        iiii = iiii + 1 
    
    print("1 here")
    
    
if seq_split > 2:
    in_sq_0 = input_sequence[0][1:178] 
    in_sq_0 = torch.cat((element_to_prepend_2a3, in_sq_0, element_to_postpend_2a3))
    
    out_2a3_0 = transformer_model(in_sq_0,tgt_sequence).to(dtype=torch.float16)

    
    in_sq_1 = input_sequence[0][179:356]
    in_sq_1 = torch.cat((element_to_prepend_2a3, in_sq_1, element_to_postpend_2a3))
    
    out_2a3_1 = transformer_model(in_sq_1,tgt_sequence).to(dtype=torch.float16)

    
    in_sq_2 = input_sequence[0][357:-1]
    in_sq_2 = torch.cat((element_to_prepend_2a3, in_sq_2, element_to_postpend_2a3)) 
    
    
    padding = (179 - len(in_sq_2))
    zeros_to_append = torch.zeros(padding).to(device=device, dtype=torch.long)
    in_sq_2 = torch.cat((in_sq_2, zeros_to_append))
    
    out_2a3_2 = transformer_model(in_sq_2,tgt_sequence).to(dtype=torch.float16)
    
    ##############
    in_sq_0 = input_sequence[0][1:178] 
    in_sq_0 = torch.cat((element_to_prepend_dms, in_sq_0, element_to_postpend_dms))
    
    out_dms_0 = transformer_model(in_sq_0,tgt_sequence).to(dtype=torch.float16)

    
    in_sq_1 = input_sequence[0][179:356]
    in_sq_1 = torch.cat((element_to_prepend_dms, in_sq_1, element_to_postpend_dms))
    
    out_dms_1 = transformer_model(in_sq_1,tgt_sequence).to(dtype=torch.float16)

    
    in_sq_2 = input_sequence[0][357:-1]
    in_sq_2 = torch.cat((element_to_prepend_dms, in_sq_2, element_to_postpend_dms)) 
    
    
    padding = (179 - len(in_sq_2))
    zeros_to_append = torch.zeros(padding).to(device=device, dtype=torch.long)
    in_sq_2 = torch.cat((in_sq_2, zeros_to_append))
    
    out_dms_2 = transformer_model(in_sq_2,tgt_sequence).to(dtype=torch.float16)
    
    print(f'{out_dms_0.size()} {out_dms_1.size()} {out_dms_2.size()} {id_minn} {id_maxx} {id_maxx - id_minn}')
    
    iiii = 0
    for iii in range(id_minn, id_minn + 178):
        sample_df.at[iii, 'reactivity_2A3_MaP'] = out_2a3_0.tolist()[0][iiii]  
        sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms_0.tolist()[0][iiii]
        iiii = iiii + 1  
    
    iiii = 0
    for iii in range(id_minn + 179, id_minn + 179 + 178):
        sample_df.at[iii, 'reactivity_2A3_MaP'] = out_2a3_1.tolist()[0][iiii]  
        sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms_1.tolist()[0][iiii]
        iiii = iiii + 1  

    iiii = 0
    for iii in range(id_minn + 179 + 179, id_maxx):
        sample_df.at[iii, 'reactivity_2A3_MaP'] = out_2a3_2.tolist()[0][iiii]  
        sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms_2.tolist()[0][iiii]
        iiii = iiii + 1  
    print("here")
        
        
#out_2a3 = transformer_model(in_sq_1,tgt_sequence).to(dtype=torch.float16)
#print(out_2a3)
'''

In [ ]:
#input_sequence[0][179:-1]
#index
#print(f'{index} {id_minn} {id_maxx} {id_maxx - id_minn} {459 - 179*2} {179*2}')

In [ ]:
print("DONe")
sample_df['id'] = sample_df['id'].astype(int)
columns_order = ['id','reactivity_DMS_MaP','reactivity_2A3_MaP']
out_sample_df = sample_df[columns_order]
out_sample_df.to_csv(SUBMISSION_PATH, index=False,float_format='%.4f')

In [ ]:
#sample_df[0:200]

In [ ]:
#test_df['Seq_Length'] = test_df['2A3_sequence'].str.len()

In [ ]:
#print(test_df['Seq_Length'].unique())

In [ ]:
#len(test_df['2A3_sequence'][1343821])
#len(test_df['2A3_sequence'][335823])



In [ ]:
#out_2a3_0.tolist()[0][9]


In [ ]:
#sample_df.at[1343821, 'reactivity_2A3_MaP']